Treat y as a progression index: higher values ≈ greater deterioration risk (e.g., rising HbA1c/complications risk). In production, a clinic would train on real outcomes; here, we map the same mechanics to a safe, open dataset.

Users & flow.

Triage Nurse opens a dashboard sorted by predicted progression (descending).
ML Service hosts /predict that the dashboard calls for each patient.
MLOps Team (you) owns training, packaging, testing, and releasing the service.
_____

Non-functional constraints.

Portability: Docker image must be self-contained.
Observability: Return JSON errors on bad input.
Reproducibility: Same code + GitHub Actions should retrain and rebuild deterministically.
_______

Iteration plan.

Iteration 1 (v0.1) — Baseline: simple StandardScaler + LinearRegression. Report RMSE on a held-out split. Ship a working API & Docker image.
Iteration 2 (v0.2) — Improvement: try Ridge/RandomForestRegressor or better preprocessing (feature scaling/selection), plus calibration of the score if you convert to a “high-risk” flag (e.g., threshold on predicted progression). Show metric deltas (RMSE; if you add a flag, also precision/recall at a threshold) in CHANGELOG.md.

In [1]:
#Import dataset
from sklearn.datasets import load_diabetes
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
RANDOM_STATE = 42

Xy = load_diabetes(as_frame=True)

X = Xy.frame.drop(columns=["target"])

#Act as a "progresion index" (Higher = worse)
y = Xy.frame["target"]


In [3]:
X.head(5)

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641


In [4]:
y.head(5)

0    151.0
1     75.0
2    141.0
3    206.0
4    135.0
Name: target, dtype: float64

Rename columns to clarify value importance

In [5]:
X = X.rename(columns={"bp": "Avg Blood pressure",
                  "s1":"Serum Cholesterol", 
                  "s2":"Low-density lipoproteins",
                  "s3":"High-density lipo",
                  "s4":"Total cholesterol",
                  "s5":"Triglycerides level",
                  "s6":"Blood sugar"})

In [6]:
X.columns

Index(['age', 'sex', 'bmi', 'Avg Blood pressure', 'Serum Cholesterol',
       'Low-density lipoproteins', 'High-density lipo', 'Total cholesterol',
       'Triglycerides level', 'Blood sugar'],
      dtype='object')

In [7]:
#Apllying 80 20 split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=RANDOM_STATE)

print(f"Traning data size:{len(X_train)}\nTesting data size:{len(X_test)}")

#Applying Standard Scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

#Fit the model (Linear Regression)
model = LinearRegression()

model.fit(X_train_scaled, y_train)

#Predict testing data
predition = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, predition)

print(f"Mean Squared Error:{mse:.2f}\n"+"_"*30)

Traning data size:309
Testing data size:133
Mean Squared Error:2872.01
______________________________
